In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

1000it [00:05, 199.27it/s]
1000it [00:03, 275.66it/s]


In [14]:
len(train_ds)

1000

In [15]:
train_ds

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x1a20cae470>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [17]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

01/28/2019 22:34:33 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.
100%|██████████| 1000/1000 [00:00<00:00, 82576.42it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[[259,  98, 111,  ..., 261, 261, 261],
           [259,  68,  98,  ..., 261, 261, 261],
           [259, 104,  98,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  98, 111,  ..., 261, 261, 261],
           [259, 115, 102,  ..., 261, 261, 261],
           [259, 105, 106,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259, 117, 112,  ..., 261, 261, 261],
           [259,  98, 260,  ..., 261, 261, 261],
           [259, 100, 112,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          ...,
  
          [[259,  35

In [23]:
batch["tokens"]["tokens"]

tensor([[[259,  98, 111,  ..., 261, 261, 261],
         [259,  68,  98,  ..., 261, 261, 261],
         [259, 104,  98,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  98, 111,  ..., 261, 261, 261],
         [259, 115, 102,  ..., 261, 261, 261],
         [259, 105, 106,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259, 117, 112,  ..., 261, 261, 261],
         [259,  98, 260,  ..., 261, 261, 261],
         [259, 100, 112,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[259,  35, 260,  ..., 261, 261, 261],
         [259, 108, 111,  ..., 261, 261, 261]

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 10, 50])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

01/28/2019 22:34:34 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[[259,  98, 111,  ..., 261, 261, 261],
          [259,  68,  98,  ..., 261, 261, 261],
          [259, 104,  98,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  98, 111,  ..., 261, 261, 261],
          [259, 115, 102,  ..., 261, 261, 261],
          [259, 105, 106,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259, 117, 112,  ..., 261, 261, 261],
          [259,  98, 260,  ..., 261, 261, 261],
          [259, 100, 112,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         ...,
 
         [[259,  35, 260,  ..., 261, 261, 261],
        

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],


In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 0.0583,  0.0099,  0.1624, -0.0822, -0.1100, -0.0373],
        [-0.1760,  0.0531,  0.0596,  0.0305,  0.1295, -0.0296],
        [-0.0013,  0.0626,  0.1495, -0.0529,  0.1343, -0.1455],
        [ 0.0787,  0.0437,  0.1275, -0.1046, -0.0256, -0.0839],
        [ 0.0173,  0.1122,  0.0885,  0.0356,  0.0350, -0.0370],
        [-0.1155,  0.0595,  0.1499, -0.2048,  0.0343, -0.0538],
        [-0.0552,  0.0340,  0.0492,  0.0453,  0.0781, -0.0915],
        [ 0.1211,  0.1789, -0.0034, -0.0561,  0.0599,  0.0239],
        [-0.0163,  0.0948,  0.0438,  0.2317,  0.1441, -0.0012],
        [-0.0378,  0.0453,  0.0443,  0.0768,  0.0850, -0.1216],
        [-0.0846, -0.1867,  0.0674, -0.1114,  0.0244, -0.2157],
        [-0.0031,  0.0181, -0.0608,  0.0348,  0.0599, -0.0244],
        [ 0.0253,  0.0801,  0.1124, -0.0218,  0.1567, -0.1301],
        [-0.0355,  0.0671,  0.0126, -0.0990,  0.0191, -0.1907],
        [-0.1806,  0.0501,  0.0078,  0.0651,  0.1327, -0.2335],
        [ 0.0969, -0.0093, -0.1086, -0.0

In [36]:
model(**batch)

{'class_logits': tensor([[-3.1902e-02, -1.6075e-02, -2.3121e-03, -1.9407e-01, -7.1642e-02,
          -2.2744e-01],
         [-1.6929e-01, -5.3192e-03, -4.5447e-02,  2.4112e-02,  2.1752e-02,
          -7.0435e-02],
         [-8.1644e-02,  4.6716e-03,  1.9318e-01, -1.1852e-02,  1.5307e-01,
          -1.5912e-01],
         [ 3.3835e-03,  1.2315e-01,  1.8093e-01, -1.3587e-01,  1.4719e-01,
          -2.1003e-01],
         [-1.3152e-02,  3.8561e-03,  1.4739e-01, -4.2846e-02,  1.1857e-02,
          -3.1865e-02],
         [-2.5713e-02,  5.2219e-02,  2.5453e-01, -6.4884e-02,  1.6502e-02,
          -5.2283e-02],
         [-8.0085e-02,  3.1069e-02,  1.1451e-01, -1.5607e-02,  2.1013e-01,
          -6.9882e-02],
         [-1.6235e-02,  1.8442e-01,  5.4850e-02,  4.5170e-02,  7.6114e-02,
          -1.1963e-01],
         [ 1.1463e-02, -1.8578e-01,  1.7596e-01,  1.3581e-01,  8.2017e-02,
          -1.1543e-01],
         [-8.8497e-02,  1.4988e-01,  1.7717e-01,  5.6029e-02,  5.1307e-02,
          -2.5562e

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6976, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[tensor([[ 4.9588e-05,  7.1738e-05,  4.2429e-05,  ..., -1.7585e-05,
           3.1004e-05,  1.3655e-05],
         [-7.9350e-05, -5.1400e-05, -8.4009e-05,  ...,  8.4053e-05,
          -4.4476e-05,  1.5511e-05],
         [-1.1336e-04, -1.9516e-05, -8.0087e-05,  ...,  6.3214e-05,
           7.3956e-05,  3.2327e-05],
         ...,
         [-5.6450e-06,  1.3279e-04,  1.7213e-05,  ...,  1.0113e-05,
          -7.5189e-05,  2.0929e-04],
         [-6.5410e-06,  4.5976e-05,  1.8543e-05,  ..., -3.8072e-05,
          -2.8101e-05,  5.2363e-05],
         [ 1.3407e-04,  1.5060e-04,  6.9589e-05,  ..., -8.6204e-05,
          -8.3227e-05,  9.2680e-05]]),
 tensor([[ 2.3907e-05, -3.9479e-06,  6.8228e-06,  ..., -2.6930e-05,
           5.9513e-06,  6.4214e-07],
         [-4.5739e-06,  1.5965e-05, -6.8646e-06,  ..., -5.8241e-06,
           5.8524e-06,  1.0220e-05],
         [-4.3652e-06,  4.4715e-05, -2.4101e-05,  ..., -6.6995e-06,
          -1.3860e-05,  1.4655e-05],
         ...,
         [ 2.5698e-05, -7

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

01/28/2019 22:34:45 - INFO - allennlp.training.trainer -   Beginning training.
01/28/2019 22:34:45 - INFO - allennlp.training.trainer -   Epoch 0/1
01/28/2019 22:34:45 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1133.563904
01/28/2019 22:34:45 - INFO - allennlp.training.trainer -   Training
loss: 0.6399 ||: 100%|██████████| 16/16 [01:36<00:00,  5.83s/it]
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -                     Training |  Validation
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   loss          |     0.640  |       N/A
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1133.564  |       N/A
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   Epoch duration: 00:01:36
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:01:36
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   Epoch 1/1
01/28/2019 22:36:21 - INFO - allennlp.training.trainer -   Peak CPU mem